In [1]:
! pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 8.9 MB/s eta 0:00:00


In [2]:
! pip install qiskit_ibm_runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.8/340.8 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 12.4 MB/s eta 0:00:00


In [3]:
! pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 73.7 MB/s eta 0:00:00


In [4]:
! pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=c1d7ac653950553aa28a12c33b5c4a1db9f5498069d6dd193a4a0867ff9dfdba
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


# **Dynamic Circuits**

In [9]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import SamplerV2 as Sampler
#from qiskit_ibm_runtime.fake_provider import FakeCairo
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from collections import defaultdict

In [8]:
# Define quantum and classical registers
controls = QuantumRegister(2, name="control")
target = QuantumRegister(1, name="target")
mid_measure = ClassicalRegister(2, name="mid")
final_measure = ClassicalRegister(1, name="final")
# Create a quantum circuit incorporating the defined registers
circuit = QuantumCircuit(controls, target, mid_measure, final_measure)

def execute_quantum_trial(q_circuit, tgt_qubit, ctrl_qubits, measurement_bits):
  # Apply Hadamard gates to target and control qubits
  q_circuit.h(tgt_qubit)
  q_circuit.h(ctrl_qubits)
  # Controlled operations
  q_circuit.ccx(*ctrl_qubits, tgt_qubit)
  q_circuit.s(tgt_qubit)
  q_circuit.ccx(*ctrl_qubits, tgt_qubit)
  # Return qubits to their original state
  q_circuit.h(ctrl_qubits)
  q_circuit.h(tgt_qubit)
  # Measure the control qubits
  q_circuit.measure(ctrl_qubits, measurement_bits)

# Execute the function on the circuit
execute_quantum_trial(circuit, target, controls, mid_measure)
# Visualize the circuit
circuit.draw(output="mpl", style='iqp', cregbundle=False)

In [ ]:
def reset_control_qubits(q_circuit, ctrl_qubits, measure_bits):
  # Reset first control qubit if its corresponding measurement bit is True
  with q_circuit.if_test((measure_bits[0], True)):
    q_circuit.x(ctrl_qubits[0])
  # Reset second control qubit if its corresponding measurement bit is True
  with q_circuit.if_test((measure_bits[1], True)):
    q_circuit.x(ctrl_qubits[1])
max_trials = 2

In [ ]:
def repeat_trials(q_circuit, tgt, ctrl, mid_meas, num_trials):
  for _ in range(num_trials - 1):
    reset_control_qubits(q_circuit, ctrl, mid_meas)
    with q_circuit.if_test((mid_meas, 0b00)) as else_block:
    # Intentionally empty block for future use with negative conditions
      pass
    with else_block:
      # Reset target and repeat the trial if conditions are not met
      q_circuit.x(tgt)
      execute_quantum_trial(q_circuit, tgt, ctrl, mid_meas)
  # Measure control qubits to capture final states due to hardware constraints
  q_circuit.measure(ctrl, mid_meas)
  # Final measurement of the target qubit to confirm the desired state
  q_circuit.measure(tgt, final_measure)

# Apply the function to the circuit
repeat_trials(circuit, target, controls, mid_measure, max_trials)
# Visualize the circuit
circuit.draw(output="mpl", style='iqp', cregbundle=False)

In [ ]:
backend = GenericBackendV2()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_circuit.draw(output="mpl", style='iqp', idle_wires=False, cregbundle=False)

In [ ]:
sampler = Sampler(backend)
job = sampler.run([isa_circuit])
hardware_counts = job.result()[0]
data_mid = job.result()[0].data.mid
data_final = job.result()[0].data.final
print(f">>> Hardware counts for mid: {data_mid.get_counts()}")
print(f">>> Hardware counts for final: {data_final.get_counts()}")
merged_data = defaultdict(int)
for bs1, bs2 in zip(data_mid.get_bitstrings(), data_final.get_bitstrings()):
  merged_data[f"{bs1}-{bs2}"] += 1
print(f">>> Hardware merged data: {dict(merged_data)}")

In [ ]:
def separate_counts_by_qubit(count_data):
  success_counts = defaultdict(int)
  failure_counts = defaultdict(int)
  for outcome, count in count_data.items():
    if outcome[:2] == "00": # Check if the first two qubits are in state 00
      success_counts[outcome[-1]] += count
    else:
      failure_counts[outcome[-1]] += count
  return success_counts, failure_counts
hw_successes, hw_failures = separate_counts_by_qubit(merged_data)
expected_successes = 1 - (1 - 5/8)**max_trials
actual_successes = sum(hw_successes.values()) / sum(merged_data.values())
print(f"Expected success rate {expected_successes:5.3f}. Actual rate {actual_successes:5.3f}.")
actual_probability = hw_successes["1"] / sum(hw_successes.values())
print(f"Expected |1> probability: 0.2. Actual probability {actual_probability:5.3f}.")

# **Transpilation**

In [ ]:
from qiskit.circuit.random import random_circuit
from qiskit.circuit.library import XGate, YGate
from qiskit_ibm_runtime.fake_provider import FakeTorino, FakeOsaka
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler import InstructionProperties, PassManager
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.plugin import list_stage_plugins
from qiskit.transpiler.timing_constraints import TimingConstraints
from qiskit.transpiler.passes.scheduling import ASAPScheduleAnalysis,PadDynamicalDecoupling
from qiskit.visualization.timeline import draw, IQXStandard
from qiskit.transpiler import StagedPassManager
from qiskit.visualization import plot_circuit_layout
from qiskit.quantum_info.analysis import hellinger_fidelity
from qiskit.visualization import plot_distribution
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def transpile_scoring(circ, layout, backend):

    """
    A custom cost function that includes T1 and T2 computed during idle periods

    Parameters:
        circ (QuantumCircuit): circuit of interest
        layouts (list of lists): List of specified layouts
        backend (IBMQBackend): An IBM Quantum backend instance

    Returns:
        float: Fidelity of circ
    """

    fid = 1

    touched = set()
    dt = backend.dt
    num_qubits = backend.num_qubits

    error=0

    t1s = [backend.qubit_properties(qq).t1 for qq in range(num_qubits)]
    t2s = [backend.qubit_properties(qq).t2 for qq in range(num_qubits)]


    for item in circ._data:
        for gate in backend.operation_names:
            if item[0].name == gate:
                if (item[0].name == 'cz') or (item[0].name == 'ecr'):
                    q0 = circ.find_bit(item[1][0]).index
                    q1 = circ.find_bit(item[1][1]).index
                    fid *= 1 - backend.target[item[0].name][(q0, q1)].error
                    touched.add(q0)
                    touched.add(q1)
                elif item[0].name == 'measure':
                    q0 = circ.find_bit(item[1][0]).index
                    fid *= 1 - backend.target[item[0].name][(q0, )].error
                    touched.add(q0)

                elif item[0].name == 'delay':
                    q0 = circ.find_bit(item[1][0]).index
                    # Ignore delays that occur before gates
                    # This assumes you are in ground state and errors
                    # do not occur.
                    if q0 in touched:
                        time = item[0].duration * dt
                        fid *= 1-qubit_error(time, t1s[q0], t2s[q0])
                else:
                    q0 = circ.find_bit(item[1][0]).index
                    fid *= 1 - backend.target[item[0].name][(q0, )].error
                    touched.add(q0)

    return fid

In [ ]:
def qubit_error(time, t1, t2):
    """Compute the approx. idle error from T1 and T2
    Parameters:
        time (float): Delay time in sec
        t1 (float): T1 time in sec
        t2 (float): T2 time in sec
    Returns:
        float: Idle error
    """
    t2 = min(t1, t2)
    rate1 = 1/t1
    rate2 = 1/t2
    p_reset = 1-np.exp(-time*rate1)
    p_z = (1-p_reset)*(1-np.exp(-time*(rate2-rate1)))/2
    return p_z + p_reset

def scoring( qc, backend):
  #ayout = qc.layout.final_virtual_layout()
  layout = qc.layout.final_index_layout()
  fidelity = transpile_scoring(qc, layout, backend)
  score = 1-fidelity
  return score

In [ ]:
seed = 10000
backend = FakeTorino()
num_qubits = 6
depth = 4
qc = random_circuit(num_qubits,depth,measure=False, seed=seed)
qc.draw('mpl')

In [ ]:
circuit_depths = {
    'opt_lv_0': None,
    'opt_lv_1': None,
    'opt_lv_2': None,
    'opt_lv_3': None,
}
gate_counts = {
    'opt_lv_0': None,
    'opt_lv_1': None,
    'opt_lv_2': None,
    'opt_lv_3': None,
}

scores = {
    'opt_lv_0': None,
    'opt_lv_1': None,
    'opt_lv_2': None,
    'opt_lv_3': None,
}

In [ ]:
# Make a pass manager with our desired optimization level and backend
pm_lv0 = generate_preset_pass_manager(backend=backend, optimization_level=0, seed_transpiler=seed)

# Run for our random circuit
tr_lv0 = pm_lv0.run(qc)

# uncomment the next line to draw circuit
#tr_lv0.draw('mpl', idle_wires=False, fold=60)

In [ ]:
circuit_depths['opt_lv_0'] =tr_lv0.depth()
gate_counts['opt_lv_0'] =sum(tr_lv0.count_ops().values())
scores['opt_lv_0'] =scoring(tr_lv0, backend)
print("Optimization level 0 results")
print("====================")
print("Circuit depth:", circuit_depths['opt_lv_0'])
print("Gate count:", gate_counts['opt_lv_0'])
print("Score:", scores['opt_lv_0'])

In [ ]:
# Make a pass manager with our desired optimization level and backend
pm_lv1 = generate_preset_pass_manager(backend=backend, optimization_level=1, seed_transpiler=seed)

# Run for our random circuit
tr_lv1 = pm_lv1.run(qc)

# uncomment the next line to draw circuit
#tr_lv1.draw('mpl', idle_wires=False, fold=60)

In [ ]:
circuit_depths['opt_lv_1'] = tr_lv1.depth()
gate_counts['opt_lv_1'] = sum(tr_lv1.count_ops().values())
scores['opt_lv_1'] = scoring(tr_lv1, backend)
print("Optimization level 1 results")
print("====================")
print("Circuit depth:", circuit_depths['opt_lv_1'])
print("Gate count:", gate_counts['opt_lv_1'])
print("Score:", scores['opt_lv_1'])

In [ ]:
pm_lv2 = generate_preset_pass_manager(backend=backend, optimization_level=2,
seed_transpiler=seed)
tr_lv2 = pm_lv2.run(qc)
#tr_lv2.draw('mpl', idle_wires=False, fold=60)

In [ ]:
circuit_depths['opt_lv_2'] = tr_lv2.depth()
gate_counts['opt_lv_2'] = sum(tr_lv2.count_ops().values())
scores['opt_lv_2'] = scoring(tr_lv2, backend)
print("Optimization level 2 results")
print("====================")
print("Circuit depth:", circuit_depths['opt_lv_2'])
print("Gate count:", gate_counts['opt_lv_2'])
print("Score:", scores['opt_lv_2'])

In [ ]:
pm_lv3 = generate_preset_pass_manager(backend=backend, optimization_level=3, seed_transpiler=seed)

tr_lv3 = pm_lv3.run(qc)

# uncomment to draw circuit
tr_lv3.draw('mpl', idle_wires=False, fold=60)

In [ ]:
circuit_depths['opt_lv_3'] = tr_lv3.depth()
gate_counts['opt_lv_3'] = sum(tr_lv3.count_ops().values())
scores['opt_lv_3'] = scoring(tr_lv3, backend)
print("Optimization level 3 results")
print("====================")
print("Circuit depth:", circuit_depths['opt_lv_3'])
print("Gate count:", gate_counts['opt_lv_3'])
print("Score:", scores['opt_lv_3'])

In [ ]:
colors = ['#FF6666', '#66B2FF']
ax = ["opt_lv_0", "opt_lv_1", "opt_lv_2", "opt_lv_3"]
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12))

# Plot 1: Circuit Depth
ax1.semilogy(ax, [circuit_depths[key] for key in ax],'o-',markersize=9, color='#FF6666', label="Depth")
ax1.set_xlabel("Optimization Level", fontsize=12)
ax1.set_ylabel("Depth", fontsize=12)
ax1.set_title("Circuit Depth", fontsize=14)
ax1.legend(fontsize=10)

# Plot 2: Total Number of Gates
ax2.semilogy(ax, [gate_counts[key] for key in ax],'^-',markersize=9, color='#66B2FF', label="Counts")
ax2.set_xlabel("Optimization Level", fontsize=12)
ax2.set_ylabel("Gate Count", fontsize=12)
ax2.set_title("Gate Count", fontsize=14)
ax2.legend(fontsize=10)

# Plot 3: Score of Transpiled Circuit
ax3.semilogy(ax, [scores[key] for key in ax],'*-',markersize=9, label="Score")
ax3.set_xlabel("Optimization Level", fontsize=12)
ax3.set_ylabel("Score", fontsize=12)
ax3.set_title("Score", fontsize=14)
ax3.legend(fontsize=10)

fig.tight_layout()
plt.show()

In [ ]:
list_stage_plugins("init")
print("Plugins run by default init stage")
print("=================================")
for i in range(4):
  print(f"\nOptimization level {i}:")
pm = generate_preset_pass_manager(backend=backend, optimization_level=i,
init_method="default", seed_transpiler=1000)
for task in pm.init.to_flow_controller().tasks:
  print(" -", type(task).__name__)

In [ ]:
for option in list_stage_plugins("layout"):
  pm = generate_preset_pass_manager(backend=backend, optimization_level=3,
  layout_method=option, seed_transpiler=seed)
  qc_tr = pm.run(qc)
  score = scoring(qc_tr, backend)
  print(f"Layout method = {option}")
  print(f"Score: {score:.6f}")
  print(f"Layout: {qc_tr.layout.final_index_layout()}\n")

In [ ]:
list_stage_plugins("routing")
print("Number of each gates of transpiled circuit and the score")
print("=================================")
for i in range(4):
    print(f"\nOptimization level {i}:")
    pm = generate_preset_pass_manager(backend=backend, optimization_level=i, routing_method='basic', seed_transpiler=seed)
    qc_tr = pm.run(qc)
    score = scoring(qc_tr, backend)
    for key, value in qc_tr.count_ops().items():
        print(key, ":", value)
    print(f"Score: {score:.6f}")

In [ ]:
print("Plugins run by basic routing stage")
print("=================================")
for i in range(4):
    print(f"\nOptimization level {i}:")
    pm = generate_preset_pass_manager(backend=backend, optimization_level=i, routing_method='basic', seed_transpiler=seed)

    for controller_group in pm.routing.to_flow_controller().tasks:
        tasks = getattr(controller_group, "tasks", [])
        for task in tasks:
            print(" - " , str(type(task).__name__))
    display(pm.routing.draw())
    print(pm.run(qc).layout.final_index_layout())

In [ ]:
## process stopped due to lookahead
options = ['basic','sabre', 'stochastic']

for option in options:
    print(f"Layout option = {option}:")
    pm = generate_preset_pass_manager(backend=backend, optimization_level=3, routing_method=option, seed_transpiler=seed)
    qc_tr = pm.run(qc)
    score = scoring(qc_tr, backend)

    print(f"Score: {score:.6f}")

    for key, value in qc_tr.count_ops().items():
        print(key, ":", value)
    print("\n")

In [ ]:
list_stage_plugins("translation")
print("Number of each gates of transpiled circuit")
print("=================================")

for i in range(4):
    print(f"\nOptimization level {i}:")
    pm = generate_preset_pass_manager(backend=backend, optimization_level=i, translation_method='translator', seed_transpiler=seed)
    qc_tr = pm.run(qc)
    score = scoring(qc_tr, backend)
    for key, value in qc_tr.count_ops().items():
        print(key, ":", value)
    print(f"Score: {score:.6f}")

In [ ]:
options = ['translator', 'synthesis']

print("Number of each gates of transpiled circuit")
print("=================================")

for option in options:
    print(f"Layout option = {option}:")
    pm = generate_preset_pass_manager(backend=backend, optimization_level=3, translation_method=option, seed_transpiler=seed)
    qc_tr = pm.run(qc)
    score = scoring(qc_tr, backend)
    for key, value in qc_tr.count_ops().items():
        print(key, ":", value)
    print(f"Score: {score:.6f}")
    print("\n")

In [ ]:
tr_depths = []
tr_gate_counts = []
tr_scores = []

options = ['translator', 'synthesis']

for i in range(4):
    for option in options:
        pm = generate_preset_pass_manager(backend=backend, optimization_level=i, translation_method=option, seed_transpiler=seed)

        tr_depths.append(pm.run(qc).depth())
        tr_gate_counts.append(sum(pm.run(qc).count_ops().values()))
        tr_scores.append(scoring(pm.run(qc), backend))

In [ ]:
colors = ['#FF6666', '#66B2FF']
markers = [ '^', '*']
linestyles = ['-.', ':']

opt_list = []
for i in range(4):
    opt_list.append(f"Optimization Level {i}")

ax = opt_list
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12))

# Plot 1: Circuit Depth
for i in range(2):
    ax1.plot(ax, tr_depths[i::2], label=options[i], marker=markers[i], markersize=9, linestyle=linestyles[i], color=colors[i], linewidth=2)
ax1.set_xlabel("translation options", fontsize=12)
ax1.set_ylabel("Depth", fontsize=12)
ax1.set_title("Circuit Depth of Transpiled Circuit", fontsize=14)
ax1.legend(fontsize=10)

# Plot 2: Total Number of Gates
for i in range(2):
    ax2.plot(ax, tr_gate_counts[i::2], label=options[i], marker=markers[i], markersize=9, linestyle=linestyles[i], color=colors[i], linewidth=2)
ax2.set_xlabel("translation options", fontsize=12)
ax2.set_ylabel("# of Total Gates", fontsize=12)
ax2.set_title("Total Number of Gates of Transpiled Circuit", fontsize=14)
ax2.legend(fontsize=10)

# Plot 3: Score of Transpiled Circuit
for i in range(2):
    ax3.plot(ax, tr_scores[i::2], label=options[i], marker=markers[i],markersize=9, linestyle=linestyles[i], color=colors[i], linewidth=2)
ax3.set_xlabel("translation options", fontsize=12)
ax3.set_ylabel("Score of Transpiled Circuit", fontsize=12)
ax3.set_title("Score of Transpiled Circuit", fontsize=14)
ax3.legend(fontsize=10)

fig.tight_layout()
plt.show()

In [ ]:
tr_depths = []
tr_gate_counts = []
tr_scores = []

approximation_degree_list = np.linspace(0,1,10)

for i in range(4):
    for j in approximation_degree_list:
        pm = generate_preset_pass_manager(backend=backend, optimization_level=i, approximation_degree=j, seed_transpiler=seed)

        tr_depths.append(pm.run(qc).depth())
        tr_gate_counts.append(sum(pm.run(qc).count_ops().values()))
        tr_scores.append(scoring(pm.run(qc), backend))

In [ ]:
colors = ['#FF6666', '#FFCC66', '#99FF99', '#66B2FF']
markers = ['o', 's', '^', '*']
linestyles = ['-', '--', '-.', ':']

ax = approximation_degree_list
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 12))

# Plot 1: Circuit Depth
for i in range(4):
    ax1.plot(ax, tr_depths[i*10:i*10+10], label=f"Optimization Level {i}", marker=markers[i], markersize=9, linestyle=linestyles[i], color=colors[i], linewidth=2)
ax1.set_xlabel("Approximation Degree", fontsize=12)
ax1.set_ylabel("Depth", fontsize=12)
ax1.set_title("Circuit Depth of Transpiled Circuit", fontsize=14)
ax1.legend(fontsize=10)

# Plot 2: Total Number of Gates
for i in range(4):
    ax2.plot(ax, tr_gate_counts[i*10:i*10+10], label=f"Optimization Level {i}", marker=markers[i], markersize=9, linestyle=linestyles[i], color=colors[i], linewidth=2)
ax2.set_xlabel("Approximation Degree", fontsize=12)
ax2.set_ylabel("# of Total Gates", fontsize=12)
ax2.set_title("Total Number of Gates of Transpiled Circuit", fontsize=14)
ax2.legend(fontsize=10)

# Plot 3: Score of Transpiled Circuit
for i in range(4):
    ax3.plot(ax, tr_scores[i*10:i*10+10], label=f"Optimization Level {i}", marker=markers[i],markersize=9, linestyle=linestyles[i], color=colors[i], linewidth=2)
ax3.set_xlabel("Approximation Degree", fontsize=12)
ax3.set_ylabel("Score of Transpiled Circuit", fontsize=12)
ax3.set_title("Score of Transpiled Circuit", fontsize=14)
ax3.legend(fontsize=10)

fig.tight_layout()
plt.show()

In [ ]:
aer_sim = AerSimulator(method='statevector')
sampler = Sampler(backend=aer_sim)
sampler.options.default_shots = 20000

qc_m = qc.measure_all(inplace=False)

pm_sim = generate_preset_pass_manager(backend=aer_sim, optimization_level=3, seed_transpiler=seed)

result = sampler.run([pm_sim.run(qc_m)]).result()[0]
ideal_counts = result.data.meas.get_counts()

In [ ]:
pm_0 = generate_preset_pass_manager(backend=backend, optimization_level=3, approximation_degree=0, seed_transpiler=seed)
pm_05 = generate_preset_pass_manager(backend=backend, optimization_level=3, approximation_degree=0.5, seed_transpiler=seed)
pm_1 = generate_preset_pass_manager(backend=backend, optimization_level=3, approximation_degree=1, seed_transpiler=seed)

noisy_sampler = Sampler(backend=backend)
noisy_sampler.options.default_shots = 20000

noisy0 = noisy_sampler.run([pm_0.run(qc_m)]).result()[0].data.meas.get_counts()
noisy05 = noisy_sampler.run([pm_05.run(qc_m)]).result()[0].data.meas.get_counts()
noisy1 = noisy_sampler.run([pm_1.run(qc_m)]).result()[0].data.meas.get_counts()

In [ ]:
print("approximation_degree = 0 fidelity", round(hellinger_fidelity(ideal_counts, noisy0), 4))
print("approximation_degree = 0.5 fidelity", round(hellinger_fidelity(ideal_counts, noisy05), 4))
print("approximation_degree = 1 fidelity", round(hellinger_fidelity(ideal_counts, noisy1), 4))

In [ ]:
list_stage_plugins("scheduling")
backend_timing = backend.target.timing_constraints()
timing_constraints = TimingConstraints(
granularity=backend_timing.granularity,
min_length=backend_timing.min_length,
pulse_alignment=backend_timing.pulse_alignment,
acquire_alignment=backend_timing.acquire_alignment)

pm_asap = generate_preset_pass_manager(
optimization_level=3,
backend=backend,
timing_constraints=timing_constraints,
scheduling_method="asap",
seed_transpiler=seed,)

my_style = {
'formatter.general.fig_width': 40,
'formatter.general.fig_unit_height': 1,
}
draw(pm_asap.run(qc), style=IQXStandard(**my_style), show_idle=False, show_delays=True)

In [ ]:
pm_alap = generate_preset_pass_manager(
optimization_level=3,
backend=backend,
timing_constraints=timing_constraints,
scheduling_method="alap",
seed_transpiler=seed,
)
draw(pm_alap.run(qc), style=IQXStandard(**my_style), show_idle=False, show_delays=True)

In [ ]:
def qubit_error(time, t1, t2):
  t2 = min(t1, t2)
  rate1 = 1/t1
  rate2 = 1/t2
  p_reset = 1-np.exp(-time*rate1)
  p_z = (1-p_reset)*(1-np.exp(-time*(rate2-rate1)))/2
  return p_z + p_reset

print("Score")
print("===============")
print(f"asap: {scoring(pm_asap.run(qc), backend):.6f}")
print(f"alap: {scoring(pm_alap.run(qc), backend):.6f}")

# **AI Transpilation**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit.circuit.library import EfficientSU2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_transpiler_service.transpiler_service import TranspilerService
from qiskit_ibm_runtime import QiskitRuntimeService

In [ ]:
NUM_QUBITS = 61
circuit = EfficientSU2(NUM_QUBITS, entanglement="circular", reps=1).decompose()
print(f"Original circuit -> Depth: {circuit.depth()}, CNOTs: {circuit.num_nonlocal_gates()}")
#circuit.draw(fold=-1, output="mpl", style="iqp", scale=0.2)

In [ ]:
QiskitRuntimeService.save_account(
channel="ibm_quantum",
token="PASTE YOUR TOKEN FROM IBM QUANTUM ACCOUNT",
set_as_default=True,
overwrite=True,
)
service = QiskitRuntimeService()
transpiler_ai_false = TranspilerService(
backend_name="ibm_brisbane",
ai=False,
optimization_level=3,
)
circuit_ai_false = transpiler_ai_false.run(circuit)
print(f"Transpiled without AI -> Depth: {circuit_ai_false.depth()}, CNOTs: {circuit_ai_false.num_nonlocal_gates()}")
#circuit_ai_false.draw(fold=-1, output="mpl", scale=0.2)

In [ ]:
transpiler_ai_true = TranspilerService(
backend_name="ibm_brisbane",
ai=True,
optimization_level=3,
)
circuit_ai_true = transpiler_ai_true.run(circuit)
print(f"Transpiled with AI -> Depth: {circuit_ai_true.depth()}, CNOTs: {circuit_ai_true.num_nonlocal_gates()}")
#circuit_ai_true.draw(fold=-1, output="mpl", scale=0.2)

In [ ]:
# Transpiling locally using Qiskit SDK
service = QiskitRuntimeService()
backend = service.backend("ibm_sherbrooke")
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
# Run and compile results
num_qubits = [11, 21, 41, 61, 81]
num_cnots_local = []
num_cnots_with_ai = []
num_cnots_without_ai = []
for nq in num_qubits:
  circuit = EfficientSU2(nq, entanglement="circular", reps=1).decompose()
  # Using the Transpiler locally on Qiskit
  circuit_local = pm.run(circuit)
  # Using the transpiler service without AI
  circuit_without_ai = transpiler_ai_false.run(circuit)
  # Using the transpiler service with AI
  circuit_with_ai = transpiler_ai_true.run(circuit)
  num_cnots_local.append(circuit_local.num_nonlocal_gates())
  num_cnots_without_ai.append(circuit_without_ai.num_nonlocal_gates())
  num_cnots_with_ai.append(circuit_with_ai.num_nonlocal_gates())

plt.plot(num_qubits, num_cnots_with_ai, '.-')
plt.plot(num_qubits, num_cnots_without_ai, '.-')
plt.plot(num_qubits, num_cnots_local, '--')
plt.xlabel("Number of qubits")
plt.ylabel("CNOT count")
plt.legend(["Qiskit Transpiler Service with AI", "Qiskit Transpiler Service without AI",
"Qiskit SDK"])

# **Circuit Knitting**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_gate_map
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from circuit_knitting.cutting import generate_cutting_experiments, cut_gates

In [ ]:
# create a bell pair
bell_state = QuantumCircuit(2)
bell_state.h(0)
bell_state.cx(0,1)
bell_state.draw("mpl")
service = QiskitRuntimeService(
channel='ibm_quantum',
instance='ibm-q/open/main',
token='PASTE YOUR TOKEN FROM IBM QUANTUM ACCOUNT'
)
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibm_kyoto")

In [ ]:
layout=[122, 126]
qubit_color = []
for i in range(127):
  if i in layout:
    qubit_color.append("#ff0066")
  else:
    qubit_color.append("#6600cc")
plot_gate_map(backend, qubit_color=qubit_color, qubit_size=60, font_size=25, figsize=(8,8))

In [ ]:
# transpile the circuit
pm = generate_preset_pass_manager(backend=backend, optimization_level=1,
initial_layout=layout, seed_transpiler=0)
isa_qc = pm.run(bell_state)
# original circuit depth
isa_qc_depth = isa_qc.depth()
print(f"Transpiled circuit depth: ", isa_qc_depth)
#isa_qc.draw("mpl", scale=0.6, idle_wires=False, fold=False)

In [ ]:
# Find the indices of the distant gates
cut_indices = [
i
for i, instruction in enumerate(bell_state.data)
if {bell_state.find_bit(q)[0] for q in instruction.qubits} == {0, 1}
]
# Decompose distant CNOTs into TwoQubitQPDGate instances
qpd_circuit, bases = cut_gates(bell_state, cut_indices)
#qpd_circuit.draw("mpl", scale=0.6)

In [ ]:
observable = SparsePauliOp(["ZI"])
# Generate the sub-experiments and sampling coefficients
sub_experiments, coefficients = generate_cutting_experiments(
circuits=qpd_circuit, observables=observable.paulis, num_samples=np.inf
)

In [ ]:
# Transpile the circuit
pm = generate_preset_pass_manager(backend=backend, optimization_level=1,
initial_layout=layout, seed_transpiler=0)
isa_qpd_circuit = pm.run(sub_experiments[5])
# depth using circuit cutting
isa_qpd_depth = isa_qpd_circuit.depth()
print(f"Original circuit depth after transpile: ", isa_qc_depth)
print(f"QPD sub-experiment depth after transpile: ", isa_qpd_depth)
print(f"Number of sub-experiments:", len(sub_experiments))
isa_qpd_circuit.draw("mpl", scale=0.6, idle_wires=False, fold=False)
x = np.array([c.depth() for c in pm.run(sub_experiments)])
print(x)

In [ ]:
toffoli_layout = [122, 124, 126]
toffoli = QuantumCircuit(3)
toffoli.ccx(0, 1, 2)
#toffoli.draw("mpl")
# Transpile the circuit
pm = generate_preset_pass_manager(backend=backend, optimization_level=1,
initial_layout=toffoli_layout, seed_transpiler=0)
isa_toffoli = pm.run(toffoli)
# Calculate original circuit depth
isa_toffoli_depth = isa_toffoli.depth()
print(f"Transpiled circuit depth: ", isa_toffoli_depth)
isa_qc.draw("mpl", scale=0.6, idle_wires=False, fold=False)

In [ ]:
# Decompose the toffoli circuit
toffoli_ = toffoli.decompose()
# Find the indices of the distant gates
gates_connecting_to_cut = {0,2}
cut_indices =[
  i
  for i, instruction in enumerate(toffoli_.data)
  if {toffoli_.find_bit(q)[0] for q in instruction.qubits} == gates_connecting_to_cut
]
# Decompose distant CNOTs into TwoQubitQPDGate instances
qpd_circuit, bases = cut_gates(toffoli_, cut_indices)
qpd_circuit.draw("mpl", scale=0.6)


In [ ]:
# set the observables
observable = SparsePauliOp(["ZZZ"])
# Generate the sub-experiments and sampling coefficients
sub_experiments, coefficients = generate_cutting_experiments(
circuits=qpd_circuit, observables=observable.paulis, num_samples=np.inf
)
# Transpile the circuit
pm = generate_preset_pass_manager(backend=backend, optimization_level=1,
initial_layout=toffoli_layout, seed_transpiler=0)
isa_qpd_toffoli = pm.run(sub_experiments[0])
# Depth using circuit cutting
isa_qpd_toffoli_depth =isa_qpd_toffoli.depth()
print(f"Transpiled circuit depth: ", isa_toffoli_depth)
print(f"QPD sub-experiment depth after transpile: ", isa_qpd_toffoli_depth)
print(f"Number of sub-experiments:", len(sub_experiments))
#isa_qpd_circuit.draw("mpl", scale=0.6, idle_wires=False, fold=False)
# mean of the depth of all sub-experiments
depth_list = np.array([c.depth() for c in pm.run(sub_experiments)])
isa_qpd_toffoli_depth_mean =depth_list.mean()
print(isa_qpd_toffoli_depth_mean)

In [ ]:
# Find the indices of the distant gates
gates_connecting_to_cut_1 = {0,2} # Hint: Expected type: set {int, int}
gates_connecting_to_cut_2 = {1,2} # Hint: Expected type: set {int, int}
cut_indices = [
i
for i, instruction in enumerate(toffoli_.data)
if {toffoli_.find_bit(q)[0] for q in instruction.qubits} == gates_connecting_to_cut_1
or {toffoli_.find_bit(q)[0] for q in instruction.qubits} == gates_connecting_to_cut_2
]
# Decompose distant CNOTs into TwoQubitQPDGate instances
qpd_circuit_2, bases = cut_gates(toffoli_, cut_indices)
qpd_circuit_2.draw("mpl", scale=0.6)

In [ ]:
# set the observables
observable = SparsePauliOp(["ZZZ"])
# Generate the sub-experiments and sampling coefficients
sub_experiments_2, coefficients = generate_cutting_experiments(
circuits=qpd_circuit_2, observables=observable.paulis, num_samples=np.inf
)
# Transpile the circuit
# Note: Use optimization_level=1 and seed_transpiler=0
pm = generate_preset_pass_manager(backend=backend, optimization_level=1,
initial_layout=toffoli_layout, seed_transpiler=0)
isa_qpd_circuit_2 = pm.run(sub_experiments_2[0])
# Depth using circuit cutting
isa_qpd_toffoli_depth_2 = isa_qpd_circuit_2.depth()
print(f"QPD sub-experiment depth after transpile: ", isa_qpd_toffoli_depth_2)
print(f"Number of sub-experiments:", len(sub_experiments_2))
#isa_qpd_circuit_2.draw("mpl", scale=0.6, idle_wires=False, fold=False)
# mean of the depth of all sub-experiments
depth_list_2 = np.array([c.depth() for c in pm.run(sub_experiments_2)])
isa_qpd_toffoli_depth_2_mean = depth_list_2.mean()
print(isa_qpd_toffoli_depth_2_mean)

In [ ]:
# Number of sub-experiments
num_sub_experiments_1_cut = len(sub_experiments)
num_sub_experiments_2_cut = len(sub_experiments_2)
# Data for plotting
categories = ['Before Cutting', 'After 1 Cut', 'After 2 Cuts']
depth_values = [isa_toffoli_depth, isa_qpd_toffoli_depth_mean, isa_qpd_toffoli_depth_2_mean]
num_sub_experiments = [1, num_sub_experiments_1_cut, num_sub_experiments_2_cut]
# Create figure and axis
fig, ax1 = plt.subplots()
# Plot depth values
color = 'tab:blue'
ax1.set_xlabel('Number of Cuts')
ax1.set_ylabel('Circuit Depth', color=color)
bars = ax1.bar(categories, depth_values, color=color, alpha=0.6)
ax1.tick_params(axis='y', labelcolor=color)
# Add value labels on bars
for bar in bars:
  yval = bar.get_height()
  ax1.text(bar.get_x() + bar.get_width() / 2, yval + 1, round(yval, 2), ha='center',
  color=color, fontsize=10)
# Create a second y-axis to plot the number of subexperiments
ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel('Number of Sub-experiments', color=color)
ax2.plot(categories, num_sub_experiments, color=color, marker='o')
ax2.tick_params(axis='y', labelcolor=color)
# Add value labels on points
for i, num in enumerate(num_sub_experiments):
  ax2.text(i, num + 0.1, num, ha='center',
  color=color, fontsize=10)
# Add titles and labels
plt.title('Circuit Knitting Toolbox Results')
fig.tight_layout() # Adjust layout to make room for both y-axes
# Show plot
plt.show()

# **Qiskit Serverless**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import warnings
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal, EfficientSU2
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options
from qiskit_serverless import QiskitFunction, save_result, get_arguments, save_result,
distribute_task, distribute_qiskit_function, IBMServerlessClient, QiskitFunction
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_transpiler_service.transpiler_service import TranspilerService
from qiskit_aer import AerSimulator

In [ ]:
num_qubits = [41, 51, 61]
circuits = [EfficientSU2(nq, su2_gates=["rz","ry"], entanglement="circular", reps=1).decompose()
for nq in num_qubits]
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibm_brisbane")

In [ ]:
optimization_levels = [1,2,3]
pass_managers = [{'pass_manager': generate_preset_pass_manager(optimization_level=level,
backend=backend), 'optimization_level': level} for level in optimization_levels]
transpiler_services = [
  {'service': TranspilerService(backend_name="ibm_brisbane", ai="false",
  optimization_level=3), 'ai': False, 'optimization_level': 3},
  {'service': TranspilerService(backend_name="ibm_brisbane", ai="true",
  optimization_level=3), 'ai': True, 'optimization_level': 3}
]
configs = pass_managers + transpiler_services

In [ ]:
def transpile_parallel_local(circuit: QuantumCircuit, config):
  """Transpilation for an abstract circuit into an ISA circuit for a given config."""
  transpiled_circuit = config.run(circuit)
  return transpiled_circuit
# Run local transpilation
warnings.filterwarnings("ignore")
start = timer()
# Run transpilations locally for baseline
results = []
for circuit in circuits:
  for config in configs:
    if 'pass_manager' in config:
      results.append(transpile_parallel_local(circuit, config['pass_manager']))
    else:
      results.append(transpile_parallel_local(circuit, config['service']))

end = timer()
# Record local execution time
execution_time_local = end - start
print("Execution time locally: ", execution_time_local)

In [ ]:
serverless = IBMServerlessClient("PASTE YOUR TOKEN FROM IBM QUANTUM ACCOUNT")

In [ ]:
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_transpiler_service.transpiler_service import TranspilerService
from qiskit_serverless import get_arguments, save_result, distribute_task, get
from qiskit_ibm_runtime import QiskitRuntimeService
from timeit import default_timer as timer

In [ ]:
@distribute_task(target={"cpu": 2})
def transpile_parallel(circuit: QuantumCircuit, config):
    """Distributed transpilation for an abstract circuit into an ISA circuit for a given backend."""
    transpiled_circuit = config.run(circuit)
    return transpiled_circuit


# Get program arguments
arguments = get_arguments()
circuits = arguments.get("circuits")
backend_name = arguments.get("backend_name")

# Get backend
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.get_backend(backend_name)

# Define Configs
optimization_levels = [1, 2, 3]
pass_managers = [generate_preset_pass_manager(optimization_level=level, backend=backend) for level in optimization_levels]

transpiler_services = [
        TranspilerService(backend_name=backend.name, ai=False, optimization_level=3),
        TranspilerService(backend_name=backend.name, ai=True, optimization_level=3),
    ]

configs = pass_managers + transpiler_services

# Start process
print("Starting timer")
start = timer()

# run distributed tasks as async function
# we get task references as a return type
sample_task_references = []
for circuit in circuits:
    sample_task_references.append([transpile_parallel(circuit, config) for config in configs])


# now we need to collect results from task references
results = get([task for subtasks in sample_task_references for task in subtasks])

end = timer()

# Record execution time
execution_time_serverless = end-start
print("Execution time: ", execution_time_serverless)

save_result({
    "transpiled_circuits": results,
    "execution_time" : execution_time_serverless

})

In [ ]:
transpile_parallel_function = QiskitFunction(
title="transpile_parallel",
entrypoint="transpile_parallel.py",
working_dir="./transpile_parallel",
dependencies=["qiskit-transpiler-service"]
)
serverless.upload(transpile_parallel_function)

In [ ]:
# Get list of functions
serverless.list()
# Fetch the specific function titled "transpile_parallel"
transpile_parallel_serverless = serverless.get('transpile_parallel')
# Run the "transpile_parallel" function in the serverless environment
job = transpile_parallel_serverless.run(
circuits= circuits,
backend_name= backend
)

In [ ]:
result = job.result()
result_transpiled = result["transpiled_circuits"]
# Compare execution times:
execution_time_serverless = result["execution_time"]
from utils import plot_execution_times
plot_execution_times(execution_time_serverless, execution_time_local)

In [ ]:
from utils import process_transpiled_circuits
best_circuits, best_depths, best_methods = process_transpiled_circuits(configs, result_transpiled)

In [ ]:
for i, (circuit, depth, method) in enumerate(zip(best_circuits, best_depths, best_methods)):
  print(f"Best result for circuit {i + 1}:")
  print(f" Depth: {depth}")
  print(f" Method: {method}")

# **Quantification of Error Rates in Quantum Circuits**

In [ ]:
import os
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.circuit import Parameter
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import numpy as np
from qiskit.primitives.containers.bindings_array import BindingsArray
from itertools import product
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives.containers.observables_array import ObservablesArray
from qiskit.primitives.containers.estimator_pub import EstimatorPub
from qiskit_ibm_runtime import EstimatorV2
import lmfit
import pandas as pd

In [ ]:
os.environ['QXToken'] = 'PASTE YOUR TOKEN FROM IBM QUANTUM ACCOUNT'
QiskitRuntimeService.save_account(channel='ibm_quantum', token=os.environ['QXToken'])
service = QiskitRuntimeService()
BACKEND = service.backend('ibm_sherbrooke')
cmap = BACKEND.configuration().coupling_map
# Search a simple connected path of at least 3 qubits
def find_path(coupling_map, length):
  path = []
  for edge in coupling_map:
    if not path:
      path.append(edge[0])
      path.append(edge[1])
    elif path[-1] == edge[0]:
      path.append(edge[1])
    elif path[-1] == edge[1]:
      path.append(edge[0])
    if len(path) >= length:
      break
  return path if len(path) >= length else None
# Get a path of at least 3 qubits
PATH_ANSWER = find_path(cmap, 5)

In [ ]:
PATH_ANSWER = [0, 1, 2, 3, 4]
# Function to create pairs from a path
def make_pairs(path):
  return [(path[i], path[i+1]) for i in range(len(path)-1)]
# Function to create layers from pairs
def make_layers(pairs, coupling_map):
  layer_1 = []
  layer_2 = []
# Create a set for quick lookup of valid edges
  valid_edges = set(tuple(edge) for edge in coupling_map)
  for i, pair in enumerate(pairs):
    if tuple(pair) not in valid_edges:
      pair = tuple(reversed(pair))
    if i % 2 == 0:
      layer_1.append(pair)
    else:
      layer_2.append(pair)
  return layer_1, layer_2
# Generate pairs from PATH_ANSWER
pairs = make_pairs(PATH_ANSWER)
# Generate the layers
LAYER_1_ANSWER, LAYER_2_ANSWER = make_layers(pairs, cmap)
print(f"PATH_ANSWER: {PATH_ANSWER}")
print(f"LAYER_1_ANSWER: {LAYER_1_ANSWER}")
print(f"LAYER_2_ANSWER: {LAYER_2_ANSWER}")

In [ ]:
GATE_NAME = None
basis_gates = BACKEND.configuration().basis_gates
if 'ecr' in basis_gates:
  GATE_NAME = 'ecr'
elif 'cz' in basis_gates:
  GATE_NAME = 'cz'
elif 'cx' in basis_gates:
  GATE_NAME = 'cx'
else:
  raise ValueError("None of the required two-qubit Clifford gates are available on the backend.")
# Function to create a circuit from a layer of pairs
def create_circuit(layer, gate_name):
  # Find the maximum qubit index to define the number of qubits in the circuit
  max_qubit = max(max(pair) for pair in layer)
  circuit = QuantumCircuit(max_qubit + 1)
  for pair in layer:
    if gate_name == 'ecr':
      circuit.ecr(pair[0], pair[1])
    elif gate_name == 'cz':
      circuit.cz(pair[0], pair[1])
    elif gate_name == 'cx':
      circuit.cx(pair[0], pair[1])
  return circuit
# Create the circuits
CIRC_1_ANSWER = create_circuit(LAYER_1_ANSWER, GATE_NAME)
CIRC_2_ANSWER = create_circuit(LAYER_2_ANSWER, GATE_NAME)
print(f"LAYER_1_ANSWER: {LAYER_1_ANSWER}")
print(f"LAYER_2_ANSWER: {LAYER_2_ANSWER}")
print(f"CIRC_1_ANSWER:\n{CIRC_1_ANSWER}")
print(f"CIRC_2_ANSWER:\n{CIRC_2_ANSWER}")
print(f"GATE_NAME: {GATE_NAME}")

In [ ]:
def eplg_circuit(num_qubits: int, depth: int, layer: QuantumCircuit, qubits: tuple[int, ...]) -> QuantumCircuit:
    if depth % 2 != 0:
        raise ValueError(f"The depth must be even, got {depth}")

    def parameters():
        _depth = 0
        while True:
            for zi in range(3):
                for q in range(num_qubits):
                    yield Parameter(f"d{_depth}_q{q}_z{zi}")
            _depth += 1
    _parameters = parameters()

    circ = QuantumCircuit(num_qubits)

    def _1q_layer():
        for j in range(3):
            for k in qubits:
                circ.rz(next(_parameters), k)
            if j == 2:
                continue
            circ.sx(qubits)

    for _depth in range(depth // 2):
        circ.barrier(qubits)
        _1q_layer()
        circ.barrier(qubits)
        circ.compose(layer, inplace=True)
    circ.barrier(qubits)
    _1q_layer()
    circ.barrier(qubits)

    # Assuming mirroring is fine
    circ = circ.compose(circ.inverse())

    circ.measure_active()

    pm = generate_preset_pass_manager(basis_gates=["ecr", "sx", "rz"], optimization_level=0)
    circ = pm.run(circ)

    circ.metadata["layer_depth"] = depth
    circ.metadata["qubits"] = qubits
    circ.metadata["num_qubits"] = num_qubits

    return circ

In [ ]:
example_circuit_1 = eplg_circuit(
    num_qubits=BACKEND.num_qubits,
    depth=4,
    layer=CIRC_1_ANSWER,
    qubits=PATH_ANSWER,
)

In [ ]:
example_circuit_2 = eplg_circuit(
    num_qubits=BACKEND.num_qubits,
    depth=4,
    layer=CIRC_2_ANSWER,
    qubits=PATH_ANSWER,
)

In [ ]:
def get_clifford_rz_samples(
    circ: QuantumCircuit,
    num_samples: int,
    rng: np.random.Generator,
) -> BindingsArray:
    """Uniformly sample the Rz parameters in a `QuantumCircuit`
    from -pi, -pi/2, 0, +pi/2, +pi.
    """

    _allowed_ops = {'rz', 'sx', 'sxdg', 'ecr', 'barrier', 'measure'}
    if (_other := set(circ.count_ops().keys()) - _allowed_ops):
        raise ValueError(f"Circuit must only contain ops: {_allowed_ops}, got {_other}")

    sampled_pars = rng.integers(-2, 3, size=(num_samples, len(circ.parameters))) * (np.pi/2)

    return BindingsArray({
        tuple(circ.parameters): sampled_pars
    })

In [ ]:
num_samples = 10
ARRAY_1_ANSWER = get_clifford_rz_samples(
    circ=example_circuit_1,
    num_samples=num_samples,
    rng=np.random.default_rng(42))
ARRAY_1_ANSWER

In [ ]:
def process_fidelity_observable(num_qubits: int, qubits: tuple[int, ...]) -> SparsePauliOp:
    """Produces the all 0 projector on `qubits` tensored with the identity on all other qubits"""
    pauli_strings = []

    for p_sub_str in product(["I", "Z"], repeat=len(qubits)):
        p_str = ["I"] * num_qubits
        for qi, pi in zip(qubits, p_sub_str):
            p_str[qi] = pi
        pauli_strings.append("".join(p_str[::-1]))

    return SparsePauliOp(pauli_strings, np.ones(len(pauli_strings)) / 2**len(qubits))

In [ ]:
from qiskit.primitives.containers.observables_array import ObservablesArray

LAYER_1_SUBSYSTEMS = LAYER_1_ANSWER + [(q,) for q in set(PATH_ANSWER) - set(item for sublist in LAYER_1_ANSWER for item in sublist)]
LAYER_2_SUBSYSTEMS = LAYER_2_ANSWER + [(q,) for q in set(PATH_ANSWER) - set(item for sublist in LAYER_2_ANSWER for item in sublist)]

LAYER_1_OBSERVABLES = ObservablesArray([process_fidelity_observable(BACKEND.num_qubits, qubits) for qubits in LAYER_1_SUBSYSTEMS])
LAYER_2_OBSERVABLES = ObservablesArray([process_fidelity_observable(BACKEND.num_qubits, qubits) for qubits in LAYER_2_SUBSYSTEMS])

In [ ]:
LAYER_1_OBSERVABLES.shape == (len(LAYER_1_SUBSYSTEMS),)

In [ ]:
LAYER_2_OBSERVABLES.shape == (len(LAYER_2_SUBSYSTEMS),)

In [ ]:
rng = np.random.default_rng(42)
num_samples = 10
depths = [2, 4, 8, 16, 32, 64]
estimator_pubs = []
for depth in depths:
    for layer, name, observables in [(CIRC_1_ANSWER, "circ_1", LAYER_1_OBSERVABLES), (CIRC_2_ANSWER, "circ_2", LAYER_2_OBSERVABLES)]:
        circ = eplg_circuit(
            num_qubits=BACKEND.num_qubits,
            depth=depth,
            layer=layer,
            qubits=PATH_ANSWER,
        )
        circ.metadata["layer_idx"] = name
        bindings_array = get_clifford_rz_samples(
            circ=circ,
            num_samples=num_samples,
            rng=rng,
        )

        # We need to reshape the `ObservablesArray`s and `BindingsArray`s so they
        # can be broadcasted.
        # https://numpy.org/doc/stable/user/basics.broadcasting.html#basics-broadcasting
        estimator_pubs.append(EstimatorPub(
            circuit=circ,
            observables=observables.reshape((*observables.shape, 1)),
            parameter_values=bindings_array.reshape((1, *bindings_array.shape)),
        ))

In [ ]:
num_evs_expected = 0
for pub in estimator_pubs:
    num_evs_expected += np.prod(estimator_pubs[0].shape)
num_evs_expected

In [ ]:
estimator = EstimatorV2(backend=BACKEND)
estimator.options.resilience.measure_mitigation = False
estimator_job = estimator.run(estimator_pubs)
primitive_result = estimator_job.result()
primitive_result

In [ ]:
data_out = {
"evs": [pub_res.data.evs for pub_res in primitive_result],
"stds": [pub_res.data.stds for pub_res in primitive_result],
"result_metadata": primitive_result.metadata,
"pub_metadata": [pub_res.metadata for pub_res in primitive_result]
}
evs_at_depth = np.zeros((len(depths), data_out['evs'][0].shape[0]*2))
for i in range(len(depths)):
  layer1_evs = np.average(data_out['evs'][2*i], axis=-1)
  layer2_evs = np.average(data_out['evs'][2*i+1], axis=-1)
  evs_at_depth[i,:] = np.array([item for pair in zip(layer1_evs, layer2_evs) for item in pair])
evs_at_depth.shape

In [ ]:
ax = plt.gca()
for i in range(evs_at_depth.shape[1]):
  ax.plot(depths, evs_at_depth[:,i])
ax.set_ylabel("Ground State Population")
ax.set_xlabel("Depth")
ax.grid()

In [ ]:
import lmfit
model= lmfit.models.ExpressionModel(
            expr="a * alpha ** x + b",
            name="rb_decay")

params_dict = {
    'a': {'value': 1.0,
            'min': -1.0,
            'max': 1.0,
            },
    'b': {'value': 0.0,
            'min': -1.0,
            'max': 1.0,
            },
    'alpha': {'value': 0.9,
              'min': 0.0,
              'max': 1.0
              },
}
p = model.make_params(**params_dict)

In [ ]:
alphas = []
for i in range(evs_at_depth.shape[-1]):
    result = model.fit(evs_at_depth[:,i], p, x=depths)
    alphas.append(result.params['alpha'].value)

In [ ]:
def cal_process_fid(alpha, D=4):
    epsilon = (D-1) * (1 - alpha)/D
    Fg = 1 - epsilon
    Fp = ((D+1) * Fg -1)/D
    return Fp

In [ ]:
qubit_pairs = [item for pair in zip(LAYER_1_ANSWER, LAYER_2_ANSWER) for item in pair]
pfs = [cal_process_fid(alpha) for alpha in alphas]
df = pd.DataFrame(zip(qubit_pairs, alphas, pfs), columns=['qubit pair', 'alpha', 'process fidelity'])
df

In [ ]:
chain_lens = np.arange(2, len(pfs), 1)
chain_fids = []
for length in chain_lens:
     w = length + 1  # window size
     fid_w = max(
          np.sqrt(pfs[s]) * np.prod(pfs[s + 1 : s + w - 1]) * np.sqrt(pfs[s + w - 1]) for s in range(len(pfs) - w + 1))
     chain_fids.append(fid_w)

In [ ]:
plt.title(f"Backend: {BACKEND.name}")
plt.plot(
chain_lens,
chain_fids,
marker="o",
linestyle="-",
)
plt.xlim(0, chain_lens[-1] * 1.05)
plt.ylim(0.95 * min(chain_fids), 1)
plt.ylabel("Layer Fidelity")
plt.xlabel("Chain Length")
plt.grid()
plt.show()

In [ ]:
num_2q_gates = [length - 1 for length in chain_lens]
chain_eplgs = [
    1 - (fid ** (1 / num_2q)) for num_2q, fid in zip(num_2q_gates, chain_fids)
]
plt.title(f"Backend: {BACKEND.name}")
plt.plot(
    chain_lens,
    chain_eplgs,
    marker="o",
    linestyle="-",
)
plt.xlim(0, chain_lens[-1] * 1.05)
plt.ylabel("Error per Layered Gates")
plt.xlabel("Chain Length")
plt.grid()
plt.show()